In [1]:
from bn_helpers.get_structures_print_tools import get_nets, printNet
from benchmarking.model_evaluator import dependency_test

MODEL = "gpt-oss:latest"

nets = get_nets()
myNet = nets[0]
printNet(myNet)
print()

print(dependency_test(myNet, model=MODEL, max_tokens=1000))

Loading Netica
A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []

prompt:
 In this Bayesian Network:
A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []

Is changing the evidence of D going to change the probability of A? Why?
quiz:
 1. Is changing the evidence of D going to change the probability of A?
A. None of the above
B. Yes
C. No

2. Why are they d-connected?
A. They are d-connected through the path ['D', 'B', 'C', 'A']
B. They are d-connected through the path ['A', 'B', 'C', 'D']
C. They are d-connected through the path ['B', 'C', 'D', 'A']
D. None of the above
y_list:
 ['B', 'A']

answer:
 


ValidationError: 1 validation error for AnswerStructure
  Invalid JSON: EOF while parsing a value at line 1 column 0 [type=json_invalid, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid

In [1]:
from ollama_helper.ollama_helper import answer_this_prompt
print(answer_this_prompt("Hello, how are you?"))

{"answer":"I'm fine, thanks! How can I help you today?"}


In [5]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model=MODEL, messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

ImportError: cannot import name 'chat' from 'ollama' (unknown location)